In [1]:
import pandas as pd

import pickle

import gensim

## Import Data

In [2]:
df = pd.read_pickle('../Data/cleaned_lyrics_lem')
df.shape

(15340, 17)

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


clean_sents = list(sent_to_words(df.lyrics))

In [4]:
def bigrams(clean_sents, bi_min=100):
    bigram = gensim.models.Phrases(clean_sents, min_count=bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod, bigram

In [5]:
def trigrams(bigram, clean_sents, tri_min=70):
    trigram = gensim.models.Phrases(bigram[clean_sents], min_count=2)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return trigram_mod

In [6]:
def get_corpus(lyrics):
    """
    Get Bigram Model, Corpus, id2word mapping
    """

    clean_sents = list(sent_to_words(lyrics))
    bigram_mod, bigram = bigrams(clean_sents)
    trigram_mod = trigrams(bigram, clean_sents)
    clean_sents = [trigram_mod[bigram_mod[t]] for t in clean_sents]

    id2word = gensim.corpora.Dictionary(clean_sents)
    corpus = [id2word.doc2bow(t) for t in clean_sents]
    return corpus, id2word, clean_sents

In [7]:
corpus, id2word, clean_sents = get_corpus(df.lyrics)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
pickle_out = open('../Data/corpus_ngramed', 'wb')
pickle.dump(corpus, pickle_out)
pickle_out.close()

pickle_out = open('../Data/id2word_ngramed', 'wb')
pickle.dump(id2word, pickle_out)
pickle_out.close()

In [9]:
for t in clean_sents[:5]:
    print(f'{" ".join(t)} \n')

old chance get gonna marry_marry get act together star school put nice little flat get finance together show old chance get gonna buy stuff buy stuff new garage store car place reherse worse never see need old chance get gonna marry_marry get act together star school put nice little flat get finance together show marry_marry marry_marry marry_marry want love 

bigger everything ever done found reason staying home tonight self tonight gonna loop duplicate heart million song found reason going tonight making tonight computer putting guitar hold track come around really many interesting effect wanna try wanna try bigger everything ever done get sleep tonight feel good tonight damn feel sweet tonight neighbor complain cause got headphone get sleep tonight damn feel good tonight everything alright computer gonna save million song hope enjoy really many interesting effect wanna try wanna try understand repetetive chatter bigger everything ever done 

going back place born favorite hood belie

In [32]:
clean_sents = pd.Series(clean_sents).apply(lambda x: ' '.join(x))
clean_sents_df = pd.DataFrame(clean_sents)
df = df.merge(clean_sents_df, left_index=True, right_index=True)
df.rename(columns={0: 'lyrics'}, inplace=True)
df.head(2)

,spotify_album_uri,spotify_artist_id,artist_name,spotify_artist_uri,duration_ms,explicit,spotify_song_id,song_title,song_spotify_page,track_number,spotify_song_uri,unaltered_song_title,unaltered_artist_name,lyrics,genius_song_id,genius_song_url,genius_artist_id,lyrics
0,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,138626,False,4okEZakOVppAtP4Dawd52x,marry me,https://open.spotify.com/track/4okEZakOVppAtP4...,1,spotify:track:4okEZakOVppAtP4Dawd52x,Marry Me,Suburban Kids With Biblical Names,old chance get gonna marry marry get act toget...,861607,https://genius.com/Suburban-kids-with-biblical...,353411,old chance get gonna marry_marry get act toget...
1,spotify:album:2h6MSR1rWemHOtmurgRq3T,7kAKqNxPBkfjgdHzUIdBtI,suburban kids with biblical names,spotify:artist:7kAKqNxPBkfjgdHzUIdBtI,187106,False,2LV6sB5zTsu0R5r5kWohlD,loop duplicate my heart,https://open.spotify.com/track/2LV6sB5zTsu0R5r...,2,spotify:track:2LV6sB5zTsu0R5r5kWohlD,Loop Duplicate My Heart,Suburban Kids With Biblical Names,bigger everything ever done x found reason sta...,980120,https://genius.com/Suburban-kids-with-biblical...,353411,bigger everything ever done found reason stayi...


## Checking if the artists match across datasets

In [11]:
len(df.artist_name.unique().tolist())

403

In [12]:
pickle_in = open('../Data/indie_pop_artists', 'rb')
queried_artists = pickle.load(pickle_in)
pickle_in.close()

In [13]:
len(list(set(queried_artists)
         .intersection(set(df.unaltered_artist_name.unique().tolist()))))

403

In [15]:
df.to_pickle('../Data/n-gramed_lyrics')